## Analytic 9 Code

#### OPIM5770 | Fall 2018 | Team 4

###### This notebook contains code to generate report for analytic 9. Designed by Team 4.

In [1]:
# Import required modules
import pandas as pd
import numpy as np
import os

In [2]:
#### ESTABLISH USER THRESHOLD FOR CUTOFF OF INTEREST
RISK_CUTOFF = input("Enter any whole digit. The resulting report will be based on invoices above this threshold.")

In [3]:
# Load the BSAK_BKPF (Note: with date parsing enabled, may take some time to complete)
parse_dates = [ 'Document_Date_in_Document'
               ,'Clearing_Date'
               ,'Day_On_Which_Accounting_Document_Was_Entered']

BSAK_BKPF_DF = pd.read_csv(r'./../../src/BSAK_BKPF.csv' 
                           , sep="|"
                           , quotechar="'"
                           , low_memory=False
                           , encoding='latin1'
                           , usecols = [ 
                                     'Company_Code',
                                     'Account_Number_of_Vendor_or_Creditor',
                                     'Reference_Document_Number',
                                     'Accounting_Document_Number',
                                     'Document_Type',
                                     'Number_of_Line_Item_Within_Accounting_Document',
                                     'Document_Number_of_the_Clearing_Document',
                                     'Reference_Document_Number',
                                     'Fiscal_Year',
                                     'Document_Date_in_Document',
                                     'Clearing_Date',
                                     'Day_On_Which_Accounting_Document_Was_Entered',
                                     'Debit_Credit_Indicator',
                                     'Currency_Key',
                                     'Amount_in_Document_Currency',
                                     'Amount_in_Local_Currency',
                                     'Reverse_Document_Number'
                                     ]
                           , dtype = { 
                                     'Company_Code': str,
                                     'Account_Number_of_Vendor_or_Creditor': str,
                                     'Reference_Document_Number': str,
                                     'Accounting_Document_Number': str,
                                     'Document_Type': str, 
                                     'Number_of_Line_Item_Within_Accounting_Document':int,
                                     'Document_Number_of_the_Clearing_Document': str,
                                     'Reference_Document_Number':str,
                                     'Fiscal_Year':int,
                                     'Document_Date_in_Document':str, # REQUIRES DATE PARSING
                                     'Clearing_Date': str, # REQUIRES DATE PARSING
                                     'Day_On_Which_Accounting_Document_Was_Entered': str, #REQUIRES DATE PARSING
                                     'Debit_Credit_Indicator': str,
                                     'Currency_Key':str,
                                     'Amount_in_Document_Currency': float,
                                     'Amount_in_Local_Currency':float,
                                     'Reverse_Document_Number':str
                                   }
                          , parse_dates=parse_dates)

BSAK_BKPF_DF.rename(columns=
          {
              'Company_Code': 'COMPANY_CODE',
              'Account_Number_of_Vendor_or_Creditor': 'VENDOR_ID',
              'Reference_Document_Number': 'VENDOR_INVOICE_NUMBER',
              'Accounting_Document_Number': 'ACCOUNTING_DOC_NUM',
              'Document_Type': 'DOCUMENT_TYPE',
              'Number_of_Line_Item_Within_Accounting_Document':'LINE_ITEM_ACCT_DOC_NUM',
              'Document_Number_of_the_Clearing_Document':'CLEARING_DOC_NUM',
              'Reference_Document_Number': 'REFERENCE_DOC_NBR',
              'Fiscal_Year': 'FISCAL_YEAR',
              'Document_Date_in_Document': 'DOCUMENT_DATE',
              'Clearing_Date': 'CLEARING_DATE',
              'Day_On_Which_Accounting_Document_Was_Entered': 'DAY_DOC_ENTERED',
              'Debit_Credit_Indicator': 'DR_CR_INDICATOR',
              'Currency_Key': 'DOC_CURRENCY_INDICATOR',
              'Amount_in_Document_Currency': 'INVOICE_AMT_DOC_CURRENCY',
              'Amount_in_Local_Currency': 'INVOICE_AMT_REPORT_CURRENCY',
              'Reverse_Document_Number': 'REVERSE_DOC_NBR',
          }, inplace=True)

In [11]:
# Filter out unnecessary data types
BSAK_BKPF_DF = BSAK_BKPF_DF[BSAK_BKPF_DF['DOCUMENT_TYPE'] != 'ZP']
BSAK_BKPF_DF = BSAK_BKPF_DF[BSAK_BKPF_DF['DOCUMENT_TYPE'] != 'KZ']

In [12]:
# Load the T001 dataframe
T001_DF = pd.read_csv(r'./../../src/T001.csv',
                     usecols = [ 'Company_Code', 
                                 'Name_of_Company_Code_or_Company',
                                 'Currency_Key'],
                     dtype = {  'Company Code': str,
                                 'Name_of_Company_Code_or_Company': str ,
                                 'Currency_Key': str
                              }
                     )

T001_DF = T001_DF.rename(columns= {
                    'Company_Code': 'COMPANY_CODE',
                    'Name_of_Company_Code_or_Company': 'COMPANY_NAME',
                    'Currency_Key': 'REPORT_CURRENCY_INDICATOR',
                    })

In [13]:
# Perform join operation
joinDF = pd.merge(BSAK_BKPF_DF,
                  T001_DF,
                 left_on='COMPANY_CODE',
                 right_on='COMPANY_CODE',
                 how='left')

In [14]:
# Add requested computed fields
joinDF['DAY_OF_WEEK'] = joinDF['DAY_DOC_ENTERED'].dt.weekday

In [15]:
# Filter the result based on the users selection
report = joinDF[joinDF['INVOICE_AMT_REPORT_CURRENCY'] >= int(RISK_CUTOFF)]

In [16]:
# Sort the result
finalReport = report.sort_values('INVOICE_AMT_REPORT_CURRENCY', ascending=True)

In [17]:
# Export the resulting report
finalReport.to_csv(r'./../output/A9_Report.csv', index=False)